In [1]:
# macro do ipython para rederizar o matplotlib inline
%matplotlib inline

# numpy é uma biblioteca de python que 
# nos permite fazer operações matriciais e vetoriais
# facilmente, e eficientemente (até um certo tamanho)
import numpy as np

import matplotlib.pyplot as plt

## Objetivo

O objetivo desse jupyter notebook é complementar a aula teórica com uma perspectiva prática do svm.
Vamos mostrar os efeitos do hiperparâmetro do SVM. Vamos dar um exemplo de classificação textutal utilizando o svm.

### Classificação textual com SVM

Vamos utilizar o conjunto de dados 20newsgroups disponível no scikit-learn. Esse conjunto de dados possui aproximadamente 18000 newsgroups posts categorizados em 20 tópicos dividos em dois conjuntos: um para treino e outro para teste (em outras palavras, para avaliação de desempenho do modelo). A divisão entre conjunto de treino e teste basedo em mensagens postadas antes e depois uma data específica.

In [8]:
from sklearn.datasets import fetch_20newsgroups

train_20ng = fetch_20newsgroups(subset='train')
test_20ng = fetch_20newsgroups(subset='test')

Vamos verificar como são os dados.

In [9]:
print(train_20ng.data[1])

From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



Os dados são textos puros dos post contendo algums metadados.

In [11]:
train_20ng.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Como já fora mencionado, há 20 categorias distintas para os postes.

Ok!! Mas como a gente faz o algoritmo entender esses textos? A gente simplesmente passa para o algoritmos os dados dessa forma? Os modelos lineares, assim como svm, não precisam trabalhar com valores numéricos contínuos e numa mesma escala? 

De fato, nós precisamos transformar esses dados textuais em numéricos e, além disso, precisamos deixá-lo em uma mesma escala.

Não sei se vocês se lembram, mas na aula demos um exemplo de problema com texto e os atributos que sugerimos foram a frequência que cada termo/palavra ocorreu em um dado post/documento.

Essa um forma simples e eficaz de representar dados textuais na forma vetorial. E é conhecido como **bag-of-words**, em bom português ***saco de palavras***.

O scikit-learn nos oferece um modo bem simples de transforma dados textutais em **bag-of-words**, então vamos lá transformar esses dados.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

# transforma dados textuais em uma presentação vetorial usando
# bag-of-words
vec = CountVectorizer()

X_train = vec.fit_transform(train_20ng.data)
y_train = train_20ng.target

X_test = vec.transform(test_20ng.data)
y_test = test_20ng.target

print("N. de Exmplos de Treino: %d; N. de Atributos: %d" % X_train.shape)
print("N. de Exmplos de Teste: %d; N. de Atributos: %d" % X_test.shape)

N. de Exmplos de Treino: 11314; N. de Atributos: 130107
N. de Exmplos de Teste: 7532; N. de Atributos: 130107


Nossa, essa transformação gerou 130 mil atributos. É coisa demais!! Isso é por causa da grande quantidade de palavras distintas que podem existir em um vocabulário. Além disso, a maioria dos valores dos atributos para cada exemplo é zero (dados esparsos), pois a maioria das palavras não acontecem em uma grande quantidade de postes. Não vamos aprofundar muito nisso agora, mas tem formas de diminuir essa quantidade de atributos, pois nem sempre mais atributos é bom.

Agora que nós já temos os dados em uma representação que o algoritmo entendi, nós iremos rodar o SVM com kernel linear.

Mas, vai dar certo isso? Os dados são linearmente separáveis? 

Então, dados textuais nesse tipo de representação bag-of-words são conhecidos por serem linearmente separáveis, portanto, uma boa posta é utilizar métodos lineares para esse tipo de problema. E de fato, o SVM linear é um método estado-da-arte para problemas que envolvem classificação textual.

In [23]:
from sklearn.svm import LinearSVC

svm = LinearSVC(random_state=123)

svm.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=123, tol=0.0001,
     verbose=0)

Vamos avaliar o desempenho do modelo aprendido pelo SVM. Geralmente em classificação de texto é utilizado como métrica de avaliação a métrica F1, que é a média harmonica entre precisão e revocação.

In [28]:
from sklearn.metrics import f1_score

y_pred = svm.predict(X_test)
f1_score(y_test, y_pred, average="micro")

0.78584705257567711

Nada mal para uma primeira tentativa. Lembram-se do hiperpârametro **C**? 

Vamos bricar um pouco com ele para vermos se conseguimos uma melhoria em nosso resultado.

In [29]:
svm = LinearSVC(C=10, random_state=123).fit(X_train, y_train)
y_pred = svm.predict(X_test)
f1_score(y_test, y_pred, average="micro")

0.77668613913967077

In [30]:
svm = LinearSVC(C=0.1, random_state=123).fit(X_train, y_train)
y_pred = svm.predict(X_test)
f1_score(y_test, y_pred, average="micro")

0.800849707912905

In [31]:
svm = LinearSVC(C=0.01, random_state=123).fit(X_train, y_train)
y_pred = svm.predict(X_test)
f1_score(y_test, y_pred, average="micro")

0.81704726500265534

Nosso primeiro teste foi C=1, o método tinha afroxado a restrição permitindo erro na "rua". Quando aumentamos para 10 o valor de C, vimos que houve uma degração um pouco maior na performance do modelo. Isso nos da evidências que afroxar a restrição não é o caminho para melhorar a predição do modelo. Desse modo, nós tentamos diminuir o valor de C, desse modo, deixando as retrições mais rígidas e vimos que nessa direção tivemos um aumento no desempenho preditivo de nosso modelo.

Escolhemnos essa maneira de calibrar os parâmetros para elucidar o impacto do hiperparâmetro no desempenho preditivo do modelo aprendido, todavia, essa não é a melhor forma de calibrar o hiperparâmetro de um método. É melhor usar métodos menos tendênciosos para escolher os valores desses hiperparâmetros, tal como validação cruzada que veremos na aula 4 desse curso.

Além de calibrar os hiperparâmetros podemos melhorar os atributos que estamos dando para nosso métodos, veremos mais isso em aulas posteriores, porém vamos dar um exemplo agora. 

### Stopwords

Stopwords são palavras cuja a ocorrência é demasiada e não ascrecenta nenhum ou pouco valor discriminativo ao método. Por exemplo, artigos e preposições (a, o, um, uma, todavia, contudo, porém, etc...). Desse modo, vamos remover essas palavras de nosso vocabulários e ver se conseguimos melhorar o modelo em alguma coisa.

In [32]:
vec = CountVectorizer(stop_words='english')

X_train = vec.fit_transform(train_20ng.data)
y_train = train_20ng.target

X_test = vec.transform(test_20ng.data)
y_test = test_20ng.target

print("N. de Exmplos de Treino: %d; N. de Atributos: %d" % X_train.shape)
print("N. de Exmplos de Teste: %d; N. de Atributos: %d" % X_test.shape)

N. de Exmplos de Treino: 11314; N. de Atributos: 129796
N. de Exmplos de Teste: 7532; N. de Atributos: 129796


In [35]:
svm = LinearSVC(C=0.01, random_state=123).fit(X_train, y_train)
y_pred = svm.predict(X_test)
f1_score(y_test, y_pred, average="micro")

0.82342007434944253

Só isso melhorou o modelo, mesmo que pouco! Vamos remover também as palavras que ocorrem muito raramente no conjunto de treino. Vamos dizer que as palavras que aparecem ao menos em 2 posts devem permanecer, se aparecerem menos serão eliminadas.

In [47]:
vec = CountVectorizer(stop_words='english', min_df=2)

X_train = vec.fit_transform(train_20ng.data)
y_train = train_20ng.target

X_test = vec.transform(test_20ng.data)
y_test = test_20ng.target

print("N. de Exmplos de Treino: %d; N. de Atributos: %d" % X_train.shape)
print("N. de Exmplos de Teste: %d; N. de Atributos: %d" % X_test.shape)

N. de Exmplos de Treino: 11314; N. de Atributos: 56126
N. de Exmplos de Teste: 7532; N. de Atributos: 56126


Isso fez com que diminuíssimos o número de atributos de 129mil para 56mil. Além disso, talvez melhorar nosso poder preditivo, ainda ajudaremos o algoritmo a aprender mais rápido.

In [48]:
svm = LinearSVC(C=0.01, random_state=123).fit(X_train, y_train)
y_pred = svm.predict(X_test)
f1_score(y_test, y_pred, average="micro")

0.82262347318109397

É... acho que mudou nada, porém o algoritmo rodou bem mais rápido :)

## Desempenho das Formulações

Na aula eu toquei rapidamente no assunto da forma Primal e Dual do problema de otimização do SVM quando estava falando sobre os Kernels. Além das diferenças na formulação permitir o uso de Kernels (no caso da formulação Dual), é importante também levar em consideração essas diferenças na escolha da formulação no que diz respeiro desempenho computacional do algoritmo.

Vamos recapitular a forma primal:

$min\ \frac{1}{2}||w||^2$

$sujeito\ a\ y_i(w \cdot x_i + b) \geq 1$


Essa é formulinha normalzinha e bonitinha! Podemos notar que o processo de minimização da mesma apenas depende de **w**, e **w** é um vetor com dimensões dependentes do número de atributos no conjunto de dados, ou seja $w \in R^d$. Isso implica em que a formulação primal é uma boa escolha quando temos muito menos atributos que exemplos de treino.

Já formulação dual depende dos exemplos de treino, como pode ser visto na equação a seguir, além disso $a_i \in R^N$:


$\min\ \sum_{ij} \alpha_i\alpha_jy_iy_j(x_i^T \cdot x_j)$

$sujeito\ a\ y_i(\sum_{i} \alpha_iy_i(x_i^T \cdot x_j) + b) \geq 1$

Isso implica em que a formulação dual é preferida quando temos muito menos exemplos do que atributos no conjunto de treino. Por exemplo, a formulação dual seria uma boa escolha nesse problema de classificação onde tínhamos 56k atributos e apenas 11k exemplos de treino.

In [50]:
%timeit svm = LinearSVC(C=0.01, random_state=123, dual=True).fit(X_train, y_train)
y_pred = svm.predict(X_test)
f1_score(y_test, y_pred, average="micro")

1 loop, best of 3: 1.93 s per loop


0.82262347318109397

In [52]:
%timeit svm = LinearSVC(C=0.01, random_state=123, dual=False).fit(X_train, y_train)
y_pred = svm.predict(X_test)
f1_score(y_test, y_pred, average="micro")

1 loop, best of 3: 58.8 s per loop


0.82262347318109397

Note que a formulação primal para esse problema demora muitíssimo!